In [ ]:

!pip install python-igraph cairocffi


In [ ]:

from google.colab import files
import igraph as ig

uploaded = files.upload()
for filename in uploaded.keys():
    gml_path = filename
    g = ig.Graph.Read_GML(gml_path)
    print(f"Loaded graph with {g.vcount()} nodes and {g.ecount()} edges.")


In [ ]:

from igraph import plot

# Compute community structures
louvain = g.community_multilevel()
girvan = g.community_edge_betweenness().as_clustering()
label_prop = g.community_label_propagation()
walktrap = g.community_walktrap().as_clustering()

community_methods = ["Louvain", "Girvan-Newman", "Label Propagation", "Walktrap"]
all_communities = [louvain, girvan, label_prop, walktrap]


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import ipywidgets as widgets
from IPython.display import display, clear_output

layout = g.layout("fr")

# Generate color list
def get_colors(memberships):
    num_comms = max(memberships) + 1
    cmap = cm.get_cmap("tab10", num_comms)
    norm = mcolors.Normalize(vmin=0, vmax=num_comms - 1)
    colors = [mcolors.to_hex(cmap(norm(c))) for c in memberships]
    return colors

# Interactivity
label_toggle = widgets.Checkbox(value=True, description='Show Node Labels')
run_button = widgets.Button(description="Redraw Communities")
output = widgets.Output()

def draw_all_igraph(communities_list, show_labels=True):
    fig, axs = plt.subplots(2, 2, figsize=(14, 12))
    axs = axs.flatten()

    for i, comm in enumerate(communities_list):
        membership = comm.membership
        g.vs["color"] = get_colors(membership)
        g.vs["label"] = g.vs["label"] if show_labels and "label" in g.vs.attributes() else None

        plot(
            g,
            layout=layout,
            target=axs[i],
            vertex_color=g.vs["color"],
            vertex_label=g.vs["label"],
            edge_color="gray",
            bbox=(400, 400),
            margin=30
        )
        axs[i].set_title(community_methods[i])

    plt.tight_layout()
    plt.show()

def on_click(b):
    with output:
        clear_output(wait=True)
        draw_all_igraph(all_communities, show_labels=label_toggle.value)

run_button.on_click(on_click)
display(widgets.HBox([label_toggle, run_button]), output)

# Initial plot
with output:
    draw_all_igraph(all_communities, show_labels=label_toggle.value)
